# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*


## Управление браузером с Selenium: введение




### Установка и подготовка к работе

Библиотека Selenium – библиотека для управления браузером с помощью Python. Она позволяет запускать браузер и имитировать действия пользователя в нём. 

Зачем это может понадобиться? Во-первых, такое «встраивание» в браузер в большинстве случаев позволяет решить проблему с возникающими капчами и иными ограничениями, так как настройки сайта не распознают, что запрос к исходному коду страницы производится автоматически. Во-вторых, необходимость имитации действий в браузере неизбежно возникает при обработке динамических веб-страниц, где некоторые элементы (окна, графики, таблицы) появляются только при определенных действиях пользователя, например, при скроллинге или наведении мышкой. В-третьих, библиотека может быть полезна в случаях, если доступ к API сайта или базы данных получить довольно сложно, но при работе в браузере информация доступна (в таком случае можем залогиниться через Python как пользователь и потихоньку выгружать данные).

Установим библиотеку:

In [1]:
!pip install selenium

  Attempting uninstall: attrs
    Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Successfully uninstalled attrs-21.2.0
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2


**NB.** Если библиотека уже установлена, команда выше вернет большой текст с сообщениями вида `Requirement already satisfied`, если не помните, когда успели ее установить, лучше принудительно обновить версию на всякий случай:

    !pip install selenium --upgrade
    
Зачем? В новой версии библиотека `selenium` при первом запуске браузера из Python сама скачает подходящий драйвер для соединения с браузером и будет использовать его в дальнейшем, а в старых версиях будет искать файл с драйвером на компьютере и выдавать ошибку.

Сначала импортируем библиотеку полностью, проверим, что всё идёт по плану:

In [2]:
import selenium

Теперь импортируем отдельные компоненты:
    
* модуль `webdriver`, нужен непосредственно для запуска браузера через Python;
* коллекция атрибутов для поиска элементов на странице `By`;
* коллекция атрибутов для имитации нажатия клавиш `Keys`.

In [3]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Запустим браузер Chrome средствами Selenium (в модуле есть функции для разных браузеров, но лучше всего работать с Chrome или Firefox):

In [31]:
br = wd.Chrome()

Если вы исполняете код выше первый раз, это может занять некоторое время, так как функция `Chrome()` сначала скачает и установит подходящую версию драйвера для Chrome, который будет обеспечивать связь между Selenium и той версией браузера Chrome, которая установлена у вас на компьютере. В последующие разы всё должно происходить быстрее.

В результате запуска строки выше должно открыться новое окно браузера Chrome. Оно пустое и, скорее всего, с всплывающим предупреждением о том, что браузером управляет автоматизированное тестовое ПО. Это нормально, мы «имитировали» открытие браузера от лица пользователя, теперь, если мы будем в этом окне переходить по ссылкам на сайты, сайты не будут воспринимать запросы как автоматические (в отличие от запросов через модуль `requests`, из-за автоматических или слишком быстрых запросов, собственно, могут возникать те же капчи).

Открывшееся окно браузера в течение работы закрывать нельзя. Python будет там выполнять действия, а мы будем их отслеживать со стороны.
Приступим к работе.

Открываем новое окно на всю ширину экрана на случай, если какие-то элементы в маленьком окне будут мешать или накладываться друг на друга (так бывает с рекламой, всплывающими окнами и подобным):

In [32]:
br.maximize_window()

### Поиск элементов на странице

Начнём с простого примера – сайта книжного магазина «Библио-Глобус». Вообще этот сайт можно парсить и без Selenium, он пока не блокирует автоматические запросы, но зато другие сайты магазинов или онлайн-кинотеатров с похожей структурой умеют это делать. 

Отправим запрос – перейдём в открытом окне по ссылке на [главную страницу](https://www.biblio-globus.ru/) сайта:

In [33]:
br.get("https://www.biblio-globus.ru/")

В окне должна открыться главная страница сайта. Найдём на странице поле для поиска интересующего товара. Для этого нам понадобится метод `.find_element()`, он применяется к объекту *WebDriver* (у нас `br`). Вообще в Selenium есть два метода для поиска, по аналогии с методами `.find()` и `.find_all()` в BeautifulSoup:

* `.find_element()` – поиск одного элемента, возвращает один результат, если подходящих элементов несколько, возвращается первый;
* `.find_elements()` – поиск нескольких элементов, возвращает список результатов.

Искать будем по id объекта, его можем найти в исходном коде страницы или через инструменты разработчика.

In [34]:
search = br.find_element(By.ID, "SearchBooks")

В инструментах разработчика id объекта отображается с `#`, классы с `.`, в нём же можно запросить более сложный запрос `XPATH`, но об этом поговорим позже. Сейчас в `search` у нас хранится объект Selenium, к которому можно применять различные методы. Так как мы нашли поле для поиска, которое можно заполнять, мы воспользуемся методом `.send_keys()`, который введёт в это поле текст. Запросим книги по Python:

In [35]:
search.send_keys("python")

В открытом окне должен появиться текст и выпадающее меню с названиями книг. Магия! 

**Примечание.** Если мы нашли объект некорректно, например, не само поле (в HTML обычно с тэгом `<input>`), а рамочку вокруг него или раздел, внутри которого это поле находится, то есть те элементы, которые не подразумевают интерактива в виде ввода значений, в ответ на `.send_keys()` мы получим ошибку `element not interactable`.

Для активации поиска нужно найти соответствующую кнопку рядом с полем и кликнуть на неё. Но можно поступить проще (не всегда подобные кнопки удобно искать), ведь мы часто вместо кликания просто нажимаем на *Enter*!

In [36]:
search.send_keys(Keys.ENTER)

Отлично! Теперь нужно поработать с результатами поиска. Предположим, что мы пишем код – основу для какой-нибудь программы или приложения, которая позволяет пользователю по ключевым словам выполнить поиск и отсортировать результаты по какому-нибудь критерию, доступному на сайте. Например, по цене, от меньшей к большей. 

Найдём выпадающее меню с нужными опциями на странице по названию тэга.

In [37]:
options = br.find_element(By.TAG_NAME, "select")

По аналогии с предыдущим примером заполнения ввода поля, выбор опций в обычных выпадающих меню с тэгом `<select>` можно осуществить с помощью того же метода `.send_keys()`.

In [38]:
options.send_keys(" Сначала дешевле ") # обратите внимание на пробелы (так в исходном коде)

Отлично! Давайте ещё найдём поле для галочки «В наличии» и кликнем на него!

In [39]:
tick = br.find_element(By.CLASS_NAME, "custom-control-label")
tick.click()

Технически, нам достаточно извлечь исходный код страницы из открытого браузера и стандартным образом выгрузить необходимую информацию с помощью BeautifulSoup. 

In [40]:
html = br.page_source

Попробуем найти карточки с найденными товарами!

In [15]:
from bs4 import BeautifulSoup

In [41]:
# ну, а тут как всегда,
# вот сами карточки, в следующем практикуме продолжим

soup = BeautifulSoup(html)

In [42]:
# имеют класс card
# первый элемент – лишний (там не товар, но класс тоже card)

cards = soup.find_all("div", class_ = "card")[1:]

Для бодрого начала сделаем скриншот той страницы, которая сейчас открыта в окне браузера, запущенном через Selenium, используя метод .save_screenshot().

In [43]:
# сохраняем в файл test.png
br.save_screenshot("test.png")

True

Напишем функцию get_cards(), которая принимает на вход строку с кодом HTML, соответствующим странице с результатами запроса (пока это первая страница результатов), и возвращает список объектов BeautifulSoup с карточками товаров (как cards выше).

In [44]:
def get_cards(html):
    soup = BeautifulSoup(html)
    cards = soup.find_all("div", class_ = "card")
    return cards

Найдем фрагмент (объект типа WebElement в Selenium) с кнопкой, которая позволяет переходить на следующую страницу поиска, и сохраним его в переменную next_page. Найдем фрагмент с кнопкой , которая позволяет переходить на последнюю страницу поиска, и сохраним его в переменную last_page.

In [45]:
# поиск по частичному совпадению текста ссылки (» или »»)

next_page = br.find_element(By.PARTIAL_LINK_TEXT, "»")
last_page = br.find_element(By.PARTIAL_LINK_TEXT, "»»")

In [46]:

# полезные примеры – как понять, что внутри объекта WebElement
# innerHTML – какой код HTML внутри фрагмента last_page

last_page.get_attribute("innerHTML")

'<span aria-hidden="true">»»</span><span class="sr-only">Next</span>'

In [47]:
# outerHTML – какой код HTML снаружи фрагмента last_page
# в какие тэги он вложен

last_page.get_attribute("outerHTML")

'<a aria-label="Next" class="page-link" href="search?query=python&amp;page=8&amp;sort=5&amp;instock=on&amp;cat=0&amp;isdiscount="><span aria-hidden="true">»»</span><span class="sr-only">Next</span></a>'

Используя метод .get_attribute(), извлечем из last_page ссылку на последнюю страницу поиска, сохраним ее в переменную last_href.

In [48]:
last_href = last_page.get_attribute("href")

In [49]:
print(last_href)

https://www.biblio-globus.ru/catalog/search?query=python&page=8&sort=5&instock=on&cat=0&isdiscount=


Используя регулярные выражения, извлечем из last_href номер последней страницы с результатами поиска и превратим результат в целое число (тип integer).

In [25]:
import re

In [50]:
last_n = int(re.search("page=(\d+)", last_href).group(1))
print(last_n)

8


Сделаем, чтобы все страницы с результатами поиска прокликались в открытом окне браузера, а в full_res оказались все карточки с товарами.

In [51]:
full_res = cards.copy()

for i in range(0, last_n - 1):
    
    next_page = br.find_element(By.PARTIAL_LINK_TEXT, "»")
    next_page.click()
    
    br.implicitly_wait(2)
    
    html = br.page_source
    cards_to_add = get_cards(html)
    full_res.extend(cards_to_add)

In [52]:

print(len(full_res)) #  число результатов

93


In [53]:
full_res

[<div class="card" style="border:0; box-shadow:none;">
 <a class="img-wrapper-index" href="/product/11015030">
 <img alt="Быстрый доступ. Python: советы, функции, подсказки. Шпаргалка-буклет для начинающих (215х285 мм, 6 полос компактного буклета в европодвесе)" class="card-img" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=11015030.jpg&amp;width=200"/>
 </a>
 <div class="card-body" style="padding-left:0; padding-right:0;">
 <div style="height:60px; overflow:hidden; margin-bottom:10px;">
 <h5 class="card-title" style="font-size:.9rem;">Быстрый доступ. Python: советы,...<br/><span class="card-author">Успенский К.Е. </span></h5>
 <p class="card-text d-none" style="font-size:.75rem;">Компактное и удобное руководство для пользователей любого уровня...</p>
 </div>
 <div>
 <div class="price_item_wrapper"><span class="price_item_title">Цена в магазине:</span> <div class="price_item_block"><span class="price_item_new">399 ₽</span></div></div>
 </div>
 <div class="mt-3" style="displ

Напишем функцию get_info(), которая принимает на вход карточку товара и возвращает список из следующих элементов:

* название товара (самое полное);
* ссылка на файл с изображением товара;
* цена товара (в текстовом формате с указанием валюты);
* полная ссылка на страницу с полным описанием товара.

In [54]:
# самое полное название – в альтернативном тексте к изображению (атрибут alt в тэге <img>)
# ссылка на изображение – в источнике изображения (атрибут src в тэге <img>)
# далее ищем подходящие разделы и ссылки, как обычно

def get_info(item):
    
    img = item.find("img")
    name = img.get("alt")
    image_link = img.get("src")
    price = item.find("div", class_ = "price_item_block").text
    link = item.find("a", class_ = "img-wrapper-index").get("href")
        
    return name, price, link, image_link

In [55]:
full_clean = []

for item in full_res:
    if len(item.get("class")) == 1:
        full_clean.append(item)
        
print(len(full_clean)) # теперь лишнее убрано, элементов меньше

78


In [56]:
import pandas as pd

In [57]:
L = [get_info(item) for item in full_clean]
df = pd.DataFrame(L)
df.head()

,0,1,2,3
0,"Быстрый доступ. Python: советы, функции, подск...",399 ₽,/product/11015030,https://static1.bgshop.ru/imagehandler.ashx?fi...
1,Тетрадь общая с пластиковой обложкой на спирал...,419 ₽,/product/10657321,https://static1.bgshop.ru/imagehandler.ashx?fi...
2,Python глазами хакера,509 ₽,/product/10823441,https://static1.bgshop.ru/imagehandler.ashx?fi...
3,Практикум по анализу данных на языках Python и R,509 ₽,/product/10882364,https://static1.bgshop.ru/imagehandler.ashx?fi...
4,Python на практике. Войти в IT с нуля,619 ₽,/product/10983821,https://static1.bgshop.ru/imagehandler.ashx?fi...


In [58]:
# добавляем названия столбцов
df.columns = ["title", "price", "link", "image_link"]

In [59]:
df

,title,price,link,image_link
0,"Быстрый доступ. Python: советы, функции, подск...",399 ₽,/product/11015030,https://static1.bgshop.ru/imagehandler.ashx?fi...
1,Тетрадь общая с пластиковой обложкой на спирал...,419 ₽,/product/10657321,https://static1.bgshop.ru/imagehandler.ashx?fi...
2,Python глазами хакера,509 ₽,/product/10823441,https://static1.bgshop.ru/imagehandler.ashx?fi...
3,Практикум по анализу данных на языках Python и R,509 ₽,/product/10882364,https://static1.bgshop.ru/imagehandler.ashx?fi...
4,Python на практике. Войти в IT с нуля,619 ₽,/product/10983821,https://static1.bgshop.ru/imagehandler.ashx?fi...
...,...,...,...,...
73,Введение в статистическое обучение с примерами...,5629 ₽,/product/11006257,https://static1.bgshop.ru/imagehandler.ashx?fi...
74,Предварительная подготовка данных в PYTHON. То...,5629 ₽,/product/10907046,https://static1.bgshop.ru/imagehandler.ashx?fi...
75,Python: Pandas на практике,5629 ₽,/product/11031733,https://static1.bgshop.ru/imagehandler.ashx?fi...
76,Введение в статистическое обучение с примерами...,5629 ₽,/product/11006257,https://static1.bgshop.ru/imagehandler.ashx?fi...
